In [1]:
!pip install pandas requests openpyxl

In [ ]:
#importing libraries
import pandas as pd
import requests
import time
import json

API_KEY = 


In [61]:
wallets_df = pd.read_excel('/content/drive/MyDrive/Zeru Finance/Stage 2/Wallet id.xlsx')
wallets_list = wallets_df.iloc[:,0].tolist()

In [67]:
def get_wallet_features(wallet_address):
    # Fetch normal transactions count
    tx_url = f"https://api.etherscan.io/api?module=account&action=txlist&address={wallet_address}&startblock=0&endblock=99999999&sort=asc&apikey={API_KEY}"
    tx_response = requests.get(tx_url).json()

    if tx_response['status'] == '1':
        tx_count = len(tx_response['result'])
    else:
        tx_count = 0

    # Fetch ETH balance
    balance_url = f"https://api.etherscan.io/api?module=account&action=balance&address={wallet_address}&tag=latest&apikey={API_KEY}"
    balance_response = requests.get(balance_url).json()

    if balance_response['status'] == '1':
        balance_eth = int(balance_response['result']) / (10**18)
    else:
        balance_eth = 0

    # Fetch Compound v2 data for borrow and supply
    compound_url = "https://api.thegraph.com/subgraphs/name/graphprotocol/compound-v2"
    compound_query = """
    {
      account(id: "%s") {
        id
        tokens {
          symbol
          supplyBalanceUnderlying
          borrowBalanceUnderlying
        }
      }
    }
    """ % wallet_address.lower()

    compound_response = requests.post(compound_url, json={'query': compound_query})
    compound_data = json.loads(compound_response.text)

    total_borrow = 0
    total_supply = 0
    if 'data' in compound_data and compound_data['data']['account'] is not None:
        tokens = compound_data['data']['account']['tokens']
        total_borrow = sum([float(t['borrowBalanceUnderlying']) for t in tokens if t['borrowBalanceUnderlying']])
        total_supply = sum([float(t['supplyBalanceUnderlying']) for t in tokens if t['supplyBalanceUnderlying']])


    # Sleep for 0.2 sec to avoid rate limits
    time.sleep(0.2)

    return {'wallet': wallet_address, 'tx_count': tx_count, 'eth_balance': balance_eth, 'borrow': total_borrow, 'supply': total_supply}

In [69]:
wallets_features = []

In [70]:

for wallet in wallets_list:
    data = get_wallet_features(wallet)
    wallets_features.append(data)

features_df = pd.DataFrame(wallets_features)

In [71]:
for col in ['tx_count','eth_balance']:
    min_val = features_df[col].min()
    max_val = features_df[col].max()
    features_df[col+'_norm'] = (features_df[col]-min_val)/(max_val-min_val+1e-6)


In [74]:
features_df['utilisation'] = features_df['borrow'] / (features_df['supply'] + 1e-6)
features_df['utilisation_norm'] = (features_df['utilisation']-features_df['utilisation'].min())/(features_df['utilisation'].max()-features_df['utilisation'].min()+1e-6)


In [75]:
features_df['score'] = (
    features_df['tx_count_norm'] * 0.6 +
    features_df['eth_balance_norm'] * 0.4
) * 1000


In [76]:
final_df = features_df[['wallet','score']]
final_df.to_csv('/content/drive/MyDrive/Zeru Finance/Stage 2/Final_Output/wallet_risk_scores.csv', index=False)


In [77]:
print(final_df.head(5))

                                       wallet       score
0  0x0039f22efb07a647557c7c5d17854cfd6d489ef3  728.495029
1  0x06b51c6882b27cb05e712185531c1f74996dd988    1.233299
2  0x0795732aacc448030ef374374eaae57d2965c16c    0.217610
3  0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9    7.399794
4  0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae    1.233299
